# Analysis of Inverted Generational Distance
This notebook demonstrates how optimization algorithm benchmarking data can be loaded from disk, metrics (such as inverted generational distance) can be calculated for all reported populations, and the resulting data analyzed for comparisons between experiements. We demonstrate the process by comparing three algorithms against eachother on a single test problem. The resulting comparisons are displayed and it is shown how the table can be exported to latex for publication.

In [1]:
import paretobench as pb

C:\Users\chris\ParetoBench\src\paretobench\analyze_metrics.py:535: SyntaxWarning: invalid escape sequence '\m'
  return " \multicolumn{1}{c}{%d/%d/%d} " % (n_plus, n_minus, n_equal)


In [2]:
%%time
# Evaluate the IGD metric on all populations stored in three saved experiments
df = pb.eval_metrics_experiments(
    ["data/NSGAII.h5", "data/CPSMOEA.h5", "data/ParEGO.h5"],
    pb.InvertedGenerationalDistance(),
    n_procs=8,
)
df.head()

CPU times: total: 31.2 ms
Wall time: 5.89 s


,problem,fevals,run_idx,pop_idx,exp_name,exp_idx,fname,igd
0,"WFG1 (m=2,n=16,k=2)",20,0,0,NSGA-II,0,data/NSGAII.h5,2.207231
1,"WFG1 (m=2,n=16,k=2)",40,0,1,NSGA-II,0,data/NSGAII.h5,1.545975
2,"WFG1 (m=2,n=16,k=2)",60,0,2,NSGA-II,0,data/NSGAII.h5,1.530251
3,"WFG1 (m=2,n=16,k=2)",80,0,3,NSGA-II,0,data/NSGAII.h5,1.529974
4,"WFG1 (m=2,n=16,k=2)",100,0,4,NSGA-II,0,data/NSGAII.h5,1.528507


In [3]:
# Calculate aggregate statistics of the metric on each history object including individuals up until a budget of `max_feval`
# function evaluations has been exceeded. Use the Wilcoxon rank sum test to compare algorithms against ParEGO.
dfa = pb.aggregate_metrics_feval_budget(df, max_feval=300, wilcoxon_idx=2)
dfa.head()

C:\Users\chris\ParetoBench\src\paretobench\analyze_metrics.py:185: SmallSampleWarning: One or more sample arguments is too small; all returned values will be NaN. See documentation for sample size requirements.
  if ranksums(our_vals.to_numpy(), other_vals.to_numpy(), "less")[1] < wilcoxon_p:
C:\Users\chris\ParetoBench\src\paretobench\analyze_metrics.py:187: SmallSampleWarning: One or more sample arguments is too small; all returned values will be NaN. See documentation for sample size requirements.
  if ranksums(our_vals.to_numpy(), other_vals.to_numpy(), "greater")[1] < wilcoxon_p:


igd            \
                                                            mean       std   
problem               exp_idx fname           exp_name                       
WFG1 (k=2, m=2, n=16) 0       data/NSGAII.h5  NSGA-II   1.472478  0.284833   
                      1       data/CPSMOEA.h5 CPS-MOEA  1.359804  0.014544   
                      2       data/ParEGO.h5  ParEGO    1.318559  0.004923   
WFG1 (k=2, m=2, n=32) 0       data/NSGAII.h5  NSGA-II   1.535811  0.304517   
                      1       data/CPSMOEA.h5 CPS-MOEA  1.369364  0.009933   

                                                                            \
                                                          median   pct_2.5   
problem               exp_idx fname           exp_name                       
WFG1 (k=2, m=2, n=16) 0       data/NSGAII.h5  NSGA-II   1.370514  1.200316   
                      1       data/CPSMOEA.h5 CPS-MOEA  1.362041  1.333089   
                      2       data/ParEGO.h5  ParEGO    1.319673  1.307050   
WFG1 (k=2, m=2, n=32) 0       data/NSGAII.h5  NSGA-II   1.430068  1.267754   
                      1       data/CPSMOEA.h5 CPS-MOEA  1.368849  1.357025   

                                                                  \
                                                        pct_97.5   
problem               exp_idx fname           exp_name             
WFG1 (k=2, m=2, n=16) 0       data/NSGAII.h5  NSGA-II   2.059161   
                      1       data/CPSMOEA.h5 CPS-MOEA  1.376491   
                      2       data/ParEGO.h5  ParEGO    1.324377   
WFG1 (k=2, m=2, n=32) 0       data/NSGAII.h5  NSGA-II   2.148538   
                      1       data/CPSMOEA.h5 CPS-MOEA  1.386730   

                                                                      \
                                                       wilcoxon_best   
problem               exp_idx fname           exp_name                 
WFG1 (k=2, m=2, n=16) 0       data/NSGAII.h5  NSGA-II          False   
                      1       data/CPSMOEA.h5 CPS-MOEA         False   
                      2       data/ParEGO.h5  ParEGO            True   
WFG1 (k=2, m=2, n=32) 0       data/NSGAII.h5  NSGA-II          False   
                      1       data/CPSMOEA.h5 CPS-MOEA         False   

                                                                      
                                                       wilcoxon_comp  
problem               exp_idx fname           exp_name                
WFG1 (k=2, m=2, n=16) 0       data/NSGAII.h5  NSGA-II              -  
                      1       data/CPSMOEA.h5 CPS-MOEA             -  
                      2       data/ParEGO.h5  ParEGO                  
WFG1 (k=2, m=2, n=32) 0       data/NSGAII.h5  NSGA-II              -  
                      1       data/CPSMOEA.h5 CPS-MOEA             -

In [4]:
# Turn the aggregate stats into a table comparing metric values for each problem between algorithms.
dfc = pb.construct_metric_comparison_table(
    dfa,
    problem_params="n",
    mean_fmt_kwargs={"precision": 3, "exp_digits": 1},
    std_fmt_kwargs={"precision": 3, "exp_digits": 1},
)
dfc

CPS-MOEA                NSGA-II  \
Problem n                                                            
WFG1    16            1.360e+0 (1.454e-2) -  1.472e+0 (2.848e-1) -   
        32            1.369e+0 (9.933e-3) -  1.536e+0 (3.045e-1) -   
        64  \cellbold 1.374e+0 (7.857e-3) =  1.494e+0 (2.471e-1) =   

                                   ParEGO  
Problem n                                  
WFG1    16  \cellbold 1.319e+0 (4.923e-3)  
        32  \cellbold 1.322e+0 (3.700e-3)  
        64                              -

In [5]:
# Demonstrate printing to latex
print(pb.comparison_table_to_latex(dfc))

\begin{tabular}{lllll}
\toprule
\multicolumn{1}{c}{\textbf{Problem }} &\multicolumn{1}{c}{\textbf{ n }}  &  \multicolumn{1}{c}{\textbf{ CPS-MOEA }}&  \multicolumn{1}{c}{\textbf{ NSGA-II }}&  \multicolumn{1}{c}{\textbf{ ParEGO }}\\
\midrule
\multirow{3}{*}{WFG1} & 16 & 1.360e+0 (1.454e-2) - & 1.472e+0 (2.848e-1) - & \cellbold 1.319e+0 (4.923e-3) \\
 & 32 & 1.369e+0 (9.933e-3) - & 1.536e+0 (3.045e-1) - & \cellbold 1.322e+0 (3.700e-3) \\
 & 64 & \cellbold 1.374e+0 (7.857e-3) $\approx$ & 1.494e+0 (2.471e-1) $\approx$ & - \\
\cline{1-5}
 \multicolumn{2}{c}{+/-/$\approx$} & \multicolumn{1}{c}{0/2/1} & \multicolumn{1}{c}{0/2/1} & \\
\bottomrule
\end{tabular}

